# load package and model

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
# sys.path

from ai_schema import *
from config_ai.backend import set_tf_config
from config_ai.utils import *
from config_ai.models.relation_classify.tf_relation_classify import TFRelationClassify
from config_ai.models.relation_classify.common import *
from config_ai.data_utils import *
from config_ai.models import load_model
from config_ai.experiments import get_model_config

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
set_tf_config()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2021-08-04 15:31:56 [INFO][backend.py:57]:setting tensorflow config...
2021-08-04 15:31:57 [INFO][backend.py:61]:current devices:
2021-08-04 15:31:57 [INFO][backend.py:62]:cpus:[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
2021-08-04 15:31:57 [INFO][backend.py:63]:gpus:[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2021-08-04 15:31:57 [INFO][backend.py:64]:setting gpu memory allow growth...
2021-08-04 15:31:57 [INFO][backend.py:67]:setting soft device placement...


In [3]:
config_path = "./config/tf_rlation_classify_config.ini"

config = read_config(config_path)
model_config = get_model_config(config)
model_config

model = TFRelationClassify(config=model_config)
model.id2label

2021-08-04 15:32:18 [INFO][utils.py:283]:parsing config with path:./config/tf_rlation_classify_config.ini
2021-08-04 15:32:18 [INFO][utils.py:294]:loading base config...
2021-08-04 15:32:18 [INFO][utils.py:283]:parsing config with path:/nfs/pony/chenhao/workspace/ConfigAI/demo/tf_base_config.ini


{'tokenizer_config': {'tokenizer_name': 'bert_word_piece',
  'tokenizer_args': {'vocabs': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt'}},
 'task_config': {'label_file_path': '/nfs/pony/chenhao/data/clue/wsc/labels.txt',
  'text_span_label_path': '/nfs/pony/chenhao/data/clue/wsc/span_labels.txt',
  'max_len': 128,
  'multi_label': False,
  'embedding_strategy': 'ENTITY_START'},
 'model_name': 'tf_relation_classify_model'}

2021-08-04 15:32:18 [INFO][core.py:42]:init model with config:
2021-08-04 15:32:18 [INFO][core.py:45]:{
    "tokenizer_config": {
        "tokenizer_name": "bert_word_piece",
        "tokenizer_args": {
            "vocabs": "/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt"
        }
    },
    "task_config": {
        "label_file_path": "/nfs/pony/chenhao/data/clue/wsc/labels.txt",
        "text_span_label_path": "/nfs/pony/chenhao/data/clue/wsc/span_labels.txt",
        "max_len": 128,
        "multi_label": false,
        "embedding_strategy": "ENTITY_START"
    },
    "model_name": "tf_relation_classify_model",
    "model_cls": "TFRelationClassify"
}
2021-08-04 15:32:18 [INFO][tf_relation_classify.py:70]:initializing tokenizer
2021-08-04 15:32:18 [INFO][tf_relation_classify.py:77]:replacing special tokens:['[S:span]', '[O:span]', '[/S]', '[/O]'] to vocabs


{0: 'true', 1: 'false'}

In [4]:
model.tokenizer_config
model.vocab_size
model.tokenizer.vocabs[10:20]


{'tokenizer_name': 'bert_word_piece',
 'tokenizer_args': {'vocabs': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt'}}

21128

['[S:span]',
 '[O:span]',
 '[/S]',
 '[/O]',
 '[unused14]',
 '[unused15]',
 '[unused16]',
 '[unused17]',
 '[unused18]',
 '[unused19]']

# process data

In [5]:
train_data_path = config['data_config']['train_data_path']
train_data = model.jload_lines(train_data_path)
f"{train_data_path}: {len(train_data)} items"


dev_data_path = config['data_config']['dev_data_path']
dev_data = model.jload_lines(dev_data_path)
f"{dev_data_path}: {len(dev_data)} items"


test_data_path = config['data_config']['test_data_path']
test_data = model.jload_lines(test_data_path)
f"{test_data_path}: {len(test_data)} items"



train_data[0]

'/nfs/pony/chenhao/data/clue/wsc/sample.jsonl: 128 items'

'/nfs/pony/chenhao/data/clue/wsc/sample.jsonl: 128 items'

'/nfs/pony/chenhao/data/clue/wsc/sample.jsonl: 128 items'

LabeledRelationClassifyExample(text='裂开的伤口涂满尘土，里面有碎石子和木头刺，我小心翼翼把它们剔除出去。', extra_text=None, text_span1=TextSpan(text='伤口', span=(3, 5), label='span', prob=1.0), text_span2=TextSpan(text='它们', span=(27, 29), label='span', prob=1.0), label=Label(name='false', prob=1.0))

In [6]:
data_manager = DataManager.get_instance(model=model, data=train_data)

In [7]:
features = data_manager.get_features()
feature = next(features)
feature

{'full_text': '裂开的[S:span]伤口[/S]涂满尘土，里面有碎石子和木头刺，我小心翼翼把[O:span]它们[/O]剔除出去。',
 'text': '裂开的[S:span]伤口[/S]涂满尘土，里面有碎石子和木头刺，我小心翼翼把[O:span]它们[/O]剔除出去。',
 'extra_text': None,
 'token_ids': [101,
  6162,
  2458,
  4638,
  10,
  839,
  1366,
  12,
  3864,
  4007,
  2212,
  1759,
  8024,
  7027,
  7481,
  3300,
  4810,
  4767,
  2094,
  1469,
  3312,
  1928,
  1173,
  8024,
  2769,
  2207,
  2552,
  5437,
  5437,
  2828,
  11,
  2124,
  812,
  13,
  1188,
  7370,
  1139,
  1343,
  511,
  102],
 'segment_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'tokens': ['[CLS]',
  '裂',
  '开',
  '的',
  '[S:span]',
  '伤',
  '口',
  '[/S]',
  '涂',
  '满',
  '尘',
  '土',
  '，',
  '里',
  '面',
  '有',
  '碎',
  '石',
  '子',
  '和',
  '木',
  '头',
  '刺',
  '，',
  '我',
  '小',
  '心',
  '翼',
  '翼',
  '把',
  '[O:span]',
  '它',
  '们',
  '[/O]',
  '剔',
  '除',
  '出',
  

In [8]:
records = data_manager.get_records(mode="train")
record = next(records)
record




{'idx': 0,
 'full_text': '裂开的[S:span]伤口[/S]涂满尘土，里面有碎石子和木头刺，我小心翼翼把[O:span]它们[/O]剔除出去。',
 'text': '裂开的[S:span]伤口[/S]涂满尘土，里面有碎石子和木头刺，我小心翼翼把[O:span]它们[/O]剔除出去。',
 'extra_text': None,
 'token_ids': [101,
  6162,
  2458,
  4638,
  10,
  839,
  1366,
  12,
  3864,
  4007,
  2212,
  1759,
  8024,
  7027,
  7481,
  3300,
  4810,
  4767,
  2094,
  1469,
  3312,
  1928,
  1173,
  8024,
  2769,
  2207,
  2552,
  5437,
  5437,
  2828,
  11,
  2124,
  812,
  13,
  1188,
  7370,
  1139,
  1343,
  511,
  102],
 'segment_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'tokens': ['[CLS]',
  '裂',
  '开',
  '的',
  '[S:span]',
  '伤',
  '口',
  '[/S]',
  '涂',
  '满',
  '尘',
  '土',
  '，',
  '里',
  '面',
  '有',
  '碎',
  '石',
  '子',
  '和',
  '木',
  '头',
  '刺',
  '，',
  '我',
  '小',
  '心',
  '翼',
  '翼',
  '把',
  '[O:span]',
  '它',
  '们',
  '[/O]',
  '剔',
  '除'

# build&compile model

In [9]:
nn_model_args = dict(**config["nn_model_config"])
nn_model_args
model.build_model(**nn_model_args)



{'pretrained_model_tag': 'bert',
 'pretrained_model_path': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12'}

2021-08-04 15:32:23 [INFO][mirrored_strategy.py:500]:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
2021-08-04 15:32:23 [INFO][tf_core.py:126]:number of devices: 1, use SINGLE scope
2021-08-04 15:32:23 [INFO][nn_models.py:56]:loading from pretrained weights: /nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/model.ckpt
2021-08-04 15:32:26 [WARNING][deprecation.py:323]:From /home/chenhao/miniconda3/envs/chenhao-py3.6-tf2.0/lib/python3.6/site-packages/tensorflow/python/util/dispatch.py:180: batch_gather (from tensorflow.python.ops.array_ops) is deprecated and will be removed after 2017-10-25.
Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.
2021-08-04 15:32:26 [INFO][tf_relation_classify.py:114]:nn model's summary:
2021-08-04 15:32:26 [INFO][layer_utils.py:192]:Model: "model_1"
2021-08-04 15:32:26 [INFO][layer_utils.py:193]:___________________________________________________________

In [10]:
compile_args = dict(**config["compile_config"])
compile_args
model.compile_model(**compile_args)


{'optimizer_name': 'adam', 'optimizer_args': {'learning_rate': 3e-05}}

2021-08-04 15:32:28 [INFO][tf_relation_classify.py:120]:compiling model...
2021-08-04 15:32:28 [INFO][tf_core.py:126]:number of devices: 1, use SINGLE scope
2021-08-04 15:32:29 [INFO][tf_relation_classify.py:137]:training model's summary:
2021-08-04 15:32:29 [INFO][layer_utils.py:192]:Model: "model_2"
2021-08-04 15:32:29 [INFO][layer_utils.py:193]:__________________________________________________________________________________________________
2021-08-04 15:32:29 [INFO][layer_utils.py:190]:Layer (type)                    Output Shape         Param #     Connected to                     
2021-08-04 15:32:29 [INFO][layer_utils.py:195]:==================================================================================================
2021-08-04 15:32:29 [INFO][layer_utils.py:190]:token_ids (InputLayer)          [(None, None)]       0                                            
2021-08-04 15:32:29 [INFO][layer_utils.py:259]:__________________________________________________________________

In [11]:
model.train_model.inputs
model.train_model.outputs
model.train_model.losses
model.train_model.metrics


[<tf.Tensor 'token_ids:0' shape=(None, None) dtype=int32>,
 <tf.Tensor 'segment_ids:0' shape=(None, None) dtype=int32>,
 <tf.Tensor 'span_idxs:0' shape=(None, 4) dtype=int32>,
 <tf.Tensor 'classify_output:0' shape=(None,) dtype=int32>]

[<tf.Tensor 'classify_layer/Identity:0' shape=(None, 2) dtype=float32>]

[<tf.Tensor 'loss_layer/Identity:0' shape=(None,) dtype=float32>]

In [12]:
model._get_dataset_info("train")
model._get_dataset_info("test")

({'token_ids': 'int32',
  'segment_ids': 'int32',
  'span_idxs': 'int32',
  'classify_output': 'int32'},
 {'token_ids': (None,),
  'segment_ids': (None,),
  'span_idxs': (4,),
  'classify_output': ()})

({'token_ids': 'int32', 'segment_ids': 'int32', 'span_idxs': 'int32'},
 {'token_ids': (None,), 'segment_ids': (None,), 'span_idxs': (4,)})

# train_model

In [13]:
train_args = dict(**config["train_config"])
train_args.update(batch_size=8, verbose=1, epochs=10)
train_args

model.train(train_data=train_data, **train_args)

{'epochs': 10, 'batch_size': 8, 'overwrite_cache': False, 'verbose': 1}

100%|██████████| 128/128 [00:00<00:00, 1235.78it/s]
2021-08-04 15:32:34 [INFO][tf_core.py:173]:train on 128 tensors


Epoch 1/10
16/16 [==============================] - 2s 122ms/step - loss: 0.7844 - accuracy: 0.4922
Epoch 2/10
16/16 [==============================] - 2s 112ms/step - loss: 0.5762 - accuracy: 0.6797
Epoch 3/10
16/16 [==============================] - 2s 112ms/step - loss: 0.4547 - accuracy: 0.8125
Epoch 4/10
16/16 [==============================] - 2s 123ms/step - loss: 0.3218 - accuracy: 0.8906
Epoch 5/10
16/16 [==============================] - 2s 112ms/step - loss: 0.2346 - accuracy: 0.9062
Epoch 6/10
16/16 [==============================] - 2s 110ms/step - loss: 0.1857 - accuracy: 0.9297
Epoch 7/10
16/16 [==============================] - 2s 109ms/step - loss: 0.1600 - accuracy: 0.9375
Epoch 8/10
16/16 [==============================] - 2s 115ms/step - loss: 0.1270 - accuracy: 0.9375
Epoch 9/10
16/16 [==============================] - 2s 111ms/step - loss: 0.1603 - accuracy: 0.9375
Epoch 10/10
16/16 [==============================] - 2s 119ms/step - loss: 0.1176 - accuracy: 0.9688

2021-08-04 15:32:58 [INFO][tf_core.py:188]:training finished


# predict&evaluate

In [16]:
to_pred = train_data[:4]
to_pred

pred = model.predict(to_pred, show_detail=True, threshold=0.999)
pred[:4]


[LabeledRelationClassifyExample(text='裂开的伤口涂满尘土，里面有碎石子和木头刺，我小心翼翼把它们剔除出去。', extra_text=None, text_span1=TextSpan(text='伤口', span=(3, 5), label='span', prob=1.0), text_span2=TextSpan(text='它们', span=(27, 29), label='span', prob=1.0), label=Label(name='false', prob=1.0)),
 LabeledRelationClassifyExample(text='裂开的伤口涂满尘土，里面有碎石子和木头刺，我小心翼翼把它们剔除出去。', extra_text=None, text_span1=TextSpan(text='尘土', span=(7, 9), label='span', prob=1.0), text_span2=TextSpan(text='它们', span=(27, 29), label='span', prob=1.0), label=Label(name='false', prob=1.0)),
 LabeledRelationClassifyExample(text='裂开的伤口涂满尘土，里面有碎石子和木头刺，我小心翼翼把它们剔除出去。', extra_text=None, text_span1=TextSpan(text='碎石子和木头刺', span=(13, 20), label='span', prob=1.0), text_span2=TextSpan(text='它们', span=(27, 29), label='span', prob=1.0), label=Label(name='true', prob=1.0)),
 LabeledRelationClassifyExample(text='这时候放在床上枕头旁边的手机响了，我感到奇怪，因为欠费已被停机两个月，现在它突然响了。', extra_text=None, text_span1=TextSpan(text='床', span=(5, 6), label='span', prob=1.0), text_span2=Text

100%|██████████| 4/4 [00:00<00:00, 2543.16it/s]
2021-08-04 15:33:47 [INFO][tf_core.py:204]:predicting with tf model...
1it [00:00, 18.72it/s]
2021-08-04 15:33:47 [INFO][utils.py:102]:function:_model_predict cost:0.055 seconds
2021-08-04 15:33:47 [INFO][utils.py:102]:function:_post_predict cost:0.002 seconds


[Label(name='false', prob=0.9941636919975281),
 Label(name='false', prob=0.9980733394622803),
 Label(name='true', prob=0.9973798394203186),
 Label(name='false', prob=0.9944508075714111)]

In [17]:
output_data = get_relation_classify_output(to_pred, pred)
output_data[:4]

[{'text': '裂开的伤口涂满尘土，里面有碎石子和木头刺，我小心翼翼把它们剔除出去。',
  'extra_text': None,
  'text_span1': {'text': '伤口', 'span': (3, 5), 'label': 'span', 'prob': 1.0},
  'text_span2': {'text': '它们', 'span': (27, 29), 'label': 'span', 'prob': 1.0},
  'label': {'name': 'false', 'prob': 1.0},
  'predict': {'name': 'false', 'prob': 0.9941636919975281}},
 {'text': '裂开的伤口涂满尘土，里面有碎石子和木头刺，我小心翼翼把它们剔除出去。',
  'extra_text': None,
  'text_span1': {'text': '尘土', 'span': (7, 9), 'label': 'span', 'prob': 1.0},
  'text_span2': {'text': '它们', 'span': (27, 29), 'label': 'span', 'prob': 1.0},
  'label': {'name': 'false', 'prob': 1.0},
  'predict': {'name': 'false', 'prob': 0.9980733394622803}},
 {'text': '裂开的伤口涂满尘土，里面有碎石子和木头刺，我小心翼翼把它们剔除出去。',
  'extra_text': None,
  'text_span1': {'text': '碎石子和木头刺',
   'span': (13, 20),
   'label': 'span',
   'prob': 1.0},
  'text_span2': {'text': '它们', 'span': (27, 29), 'label': 'span', 'prob': 1.0},
  'label': {'name': 'true', 'prob': 1.0},
  'predict': {'name': 'true', 'prob': 0.9973798394

In [18]:
true_labels = [e.label for e in to_pred]
true_labels
pred

eval_rs = eval_relation_classify(true_labels, pred)
eval_rs

[Label(name='false', prob=1.0),
 Label(name='false', prob=1.0),
 Label(name='true', prob=1.0),
 Label(name='false', prob=1.0)]

[Label(name='false', prob=0.9941636919975281),
 Label(name='false', prob=0.9980733394622803),
 Label(name='true', prob=0.9973798394203186),
 Label(name='false', prob=0.9944508075714111)]

{'detail': {'true': {'tp': 1,
   'fp': 0,
   'fn': 0,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0},
  'false': {'tp': 3,
   'fp': 0,
   'fn': 0,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0}},
 'micro': {'tp': 4,
  'fp': 0,
  'fn': 0,
  'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0},
 'macro': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}}

# save&load

In [19]:
model_path = f"./models/{model.model_name}"
save_args = dict(**config["common_config"]["save_args"])
del save_args["tf_serving_version"]

save_args
model_path


model.save(path=model_path, **save_args)




{'format': 'h5'}

'./models/tf_relation_classify_model'

2021-08-04 15:34:41 [INFO][core.py:74]:saving model to ./models/tf_relation_classify_model
2021-08-04 15:34:41 [INFO][tf_core.py:56]:saving keras model to path:./models/tf_relation_classify_model/nn_model/nn_model.h5
2021-08-04 15:34:43 [INFO][core.py:126]:save model done


In [20]:
loaded_model = load_model(path=model_path)
loaded_model

2021-08-04 15:34:45 [INFO][core.py:85]:loading model from path:./models/tf_relation_classify_model
2021-08-04 15:34:45 [INFO][core.py:42]:init model with config:
2021-08-04 15:34:45 [INFO][core.py:45]:{
    "tokenizer_config": {
        "tokenizer_name": "bert_word_piece",
        "tokenizer_args": {
            "vocabs": "/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt"
        }
    },
    "task_config": {
        "label_file_path": "/nfs/pony/chenhao/data/clue/wsc/labels.txt",
        "text_span_label_path": "/nfs/pony/chenhao/data/clue/wsc/span_labels.txt",
        "max_len": 128,
        "multi_label": false,
        "embedding_strategy": "ENTITY_START"
    },
    "model_name": "tf_relation_classify_model",
    "model_cls": "TFRelationClassify",
    "model_dict": {
        "test": {
            "model": "model_1",
            "type_info": {
                "token_ids": "int32",
                "segment_ids": "int32",
                "span_idxs": "int32"
          

In [22]:
loaded_pred = loaded_model.predict(data=to_pred, threshold=0.5)
loaded_pred

100%|██████████| 4/4 [00:00<00:00, 2229.83it/s]
2021-08-04 15:35:02 [INFO][tf_core.py:204]:predicting with tf model...
2021-08-04 15:35:02 [INFO][utils.py:102]:function:_model_predict cost:0.051 seconds
2021-08-04 15:35:02 [INFO][utils.py:102]:function:_post_predict cost:0.002 seconds


[Label(name='false', prob=0.9941636919975281),
 Label(name='false', prob=0.9980733394622803),
 Label(name='true', prob=0.9973798394203186),
 Label(name='false', prob=0.9944508075714111)]

In [24]:
tmp_data = model.jload_lines("/nfs/pony/chenhao/data/clue/wsc/test.jsonl")
tmp_data = tmp_data[:4]
tmp_data


loaded_pred = loaded_model.predict(data=tmp_data,threshold=0.5)


[RelationClassifyExample(text='杨百顺他爹是个卖豆腐的。别人叫他卖豆腐的老杨。', extra_text=None, text_span1=TextSpan(text='杨百顺', span=(0, 3), label='span', prob=1.0), text_span2=TextSpan(text='他', span=(15, 16), label='span', prob=1.0)),
 RelationClassifyExample(text='杨百顺他爹是个卖豆腐的。别人叫他卖豆腐的老杨。', extra_text=None, text_span1=TextSpan(text='杨百顺他爹', span=(0, 5), label='span', prob=1.0), text_span2=TextSpan(text='他', span=(15, 16), label='span', prob=1.0)),
 RelationClassifyExample(text='老孔说话声儿细；老窦是个急性子，当年一脚把我的鼓给踹破了。我也没输给他，回头一脚，把他的摊子也踢了，胡辣汤流了一地。', extra_text=None, text_span1=TextSpan(text='老窦', span=(8, 10), label='span', prob=1.0), text_span2=TextSpan(text='他', span=(42, 43), label='span', prob=1.0)),
 RelationClassifyExample(text='老孔说话声儿细；老窦是个急性子，当年一脚把我的鼓给踹破了。我也没输给他，回头一脚，把他的摊子也踢了，胡辣汤流了一地。', extra_text=None, text_span1=TextSpan(text='老孔', span=(0, 2), label='span', prob=1.0), text_span2=TextSpan(text='他', span=(42, 43), label='span', prob=1.0))]

100%|██████████| 4/4 [00:00<00:00, 3071.63it/s]
2021-08-04 15:35:23 [INFO][tf_core.py:204]:predicting with tf model...
2021-08-04 15:35:23 [INFO][utils.py:102]:function:_model_predict cost:0.086 seconds
2021-08-04 15:35:23 [INFO][utils.py:102]:function:_post_predict cost:0.002 seconds
